In [497]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import unicodedata
%matplotlib inline

In [546]:
from bs4 import BeautifulSoup
from urllib.request import urlopen

In [561]:
html = urlopen("http://www.htfc.com/main/a/20170619/80000072.shtml")
bsObj = BeautifulSoup(html.read(),'lxml')


In [570]:
newData = pd.DataFrame()
i = 1
flag = 0
tickdict = []
newData = pd.DataFrame()
nameList = bsObj.findAll("span", {"style":"FONT-FAMILY: 仿宋,仿宋_GB2312"})
if nameList == []:
    #print(1)
    nameList = bsObj.findAll("span", {"style":"font-family:仿宋,仿宋_GB2312"})
# if nameList == []:
#     print(1)
#     nameList = bsObj.findAll("span", {"style":"font-family:仿宋,仿宋_GB2312"})
for name in nameList:
    #print(name.get_text().strip(),1)
    #if name.get_text().strip() == "上海期货交易所:" or flag == 1: #or (name.get_text().strip()[0] == "铜" or ""):
        #if name.get_text().strip() == "上海期货交易所:":
            #flag = 1
    if name.get_text().strip() not in ["上海期货交易所:","郑州商品交易所:",
                               "大连商品交易所（交割月份的涨跌停板幅度为上一交易日结算价的6%）:",
                               "大连商品交易所:","中国金融期货交易所:"]:
        m = name.get_text().strip()
        if m.strip() not in ['','5','10']:
            m = unicodedata.normalize("NFKD", m) ## https://stackoverflow.com/questions/10993612/python-removing-xa0-from-string
            m.split(maxsplit=1)
            #print(m)
            tickdict.append(m)

NewData = pd.DataFrame(tickdict)
NewData = NewData[len(NewData) - 102:].reset_index(drop=True) #因为期货总共51个品种，所以我们仅仅需要下面的102行

s = NewData[0].apply(lambda x: x.split(':'))
##df = pd.DataFrame(s)
NewData['Code'],NewData['Limit'] = zip(*NewData[0].apply(lambda x: x.split(':',maxsplit = 1)))

NewData["Comments"] = ""
NewData['Name'] = ""

for i in range(len(NewData)):
    NewData['Limit'].iloc[i] = NewData['Limit'].iloc[i].strip()

for i in range(len(NewData)):
    NewData['Name'].iloc[i] = (NewData['Code'].iloc[i])[:-2].strip().replace(' ','')
    NewData['Code'].iloc[i] = (NewData['Code'].iloc[i])[-2:].strip()
    ##print((NewData['Tick'].iloc[i])[:-2])
    if len(NewData['Limit'].iloc[i]) > 6:
        NewData['Limit'].iloc[i],NewData['Comments'].iloc[i] = NewData['Limit'].iloc[i].split('(',maxsplit = 1)
        #if len(NewData['C'].iloc[i]) > 2:
        NewData['Comments'].iloc[i] = NewData['Comments'].iloc[i].replace('(','').replace(')','')
del NewData[0]


NewData['Margin'] = ""
NewData['Margin Comments'] = ""
# for item in list(NewData['Name']):
# #NewData[NewData['Name'] == item]
# #df.loc[df['column_name'] == some_value]
# NewData.loc[NewData['Name'] == item])
list_tick = NewData.set_index('Name').index.get_duplicates()
#NewData[NewData['Name'] == list_tick[0]].Limit.index[]
for i in range(len(list_tick)):
    pos = NewData[NewData['Name'] == list_tick[i]].Limit.index[0]
    
    NewData['Margin'].iloc[pos] = NewData[NewData['Name'] == list_tick[i]].Limit.values[1]
    NewData['Margin Comments'].iloc[pos] = NewData[NewData['Name'] == list_tick[i]].Comments.values[1]
NewData = NewData[['Name', 'Code', 'Limit','Comments','Margin', 'Margin Comments']]
NewData = NewData.iloc[:49]
NewData.to_csv('DailyContractsInformation.csv',encoding='utf_8_sig')